<a href="https://colab.research.google.com/github/LasseyMiracle/Item-truck-assignment-variant-VRP-solving-using-genetic-algorithm/blob/main/item_to_truck_assignment_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
"""
# -*- coding: utf-8 -*-
"""
Created on Sun Dec  8 16:11:59 2024

@author: Miracle K. Lassey
"""

import csv
import math
import random


def initialize_trips(distance_file, trucks_file, orders_file):
    """
    Initializes trips from CSV files, dynamically generating trip_id.

    Args:
        distance_file (str): Path to the distance CSV file.
        trucks_file (str): Path to the trucks CSV file.
        orders_file (str): Path to the orders CSV file.

    Returns:
        list: A list of trips, each represented as a dictionary.
    """

    trips = []
    trip_groups = {}  # To store trip groups based on source and destination

    # Read orders data first to group by source and destination
    with open(orders_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            source = row['Source']  # Assuming 'Source', 'Destination', 'weight', 'area' columns
            destination = row['Destination']


            trip_key = (source, destination)
            if trip_key not in trip_groups:
                trip_groups[trip_key] = []  # Create a new group if it doesn't exist
            trip_groups[trip_key].append(row)  # Add the order to the group

    # Create trips based on the groups
    trip_counter = 1
    for trip_key, orders in trip_groups.items():
        trip_id = f"trip_{trip_counter}"
        trip_counter += 1

        total_weight = sum(float(order['Weight']) for order in orders)
        total_area = sum(float(order['Area']) for order in orders)

        # Get distance from distance_file (if available)
        distance = None
        with open(distance_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row['Source'] == trip_key[0] and row['Destination'] == trip_key[1]:
                    distance = float(row['Distance'])  # Assuming 'distance' column
                    break

        trips.append({
            'trip_id': trip_id,
            'trip_distance': distance,  # Or a default value if not found
            'source': trip_key[0],
            'destination': trip_key[1],
            'trip_weight': total_weight,
            'trip_area': total_area,
            'trip_truck_type': None,  # To be determined later
            'number_truck_type': 0  # To be determined later
        })


    # Read trucks data and assign truck types to trips
    with open(trucks_file, 'r') as f:
        reader = csv.DictReader(f)
        trucks = list(reader)  # Store all truck data

        for trip in trips:
            # Find suitable truck type
            suitable_truck = None
            for truck in trucks:
                if float(truck['Weight']) >= trip['trip_weight'] and \
                   float(truck['Area']) >= trip['trip_area']:
                    suitable_truck = truck  # Store the suitable truck
                    break

            # Calculate number of trucks needed
            if suitable_truck:
                trip['trip_truck_type'] = suitable_truck['Truck_type']
                trip['number_truck_type'] = int(math.ceil(max( # Changed to int()
                    trip['trip_weight'] / float(suitable_truck['Weight']),
                    trip['trip_area'] / float(suitable_truck['Area'])
                )))
            else:
                #If no single truck can handle, find the largest truck and increase number
                largest_truck = max(trucks, key=lambda truck: float(truck['Weight']) * float(truck['Area']))
                trip['trip_truck_type'] = largest_truck['Truck_type']
                trip['number_truck_type'] = int(math.ceil(max( # Changed to int()
                    trip['trip_weight'] / float(largest_truck['Weight']),
                    trip['trip_area'] / float(largest_truck['Area'])
                )))

    return trips



def initialize_chromosome_trips(list_of_trips):
    """
    Initializes a chromosome for the trips scenario.

    Args:
        list_of_trips: A list of dictionaries, where each dictionary represents a trip
                       with 'trip_id' as a key.

    Returns:
        A list representing the chromosome.
    """
    chromosome = []
    for trip in list_of_trips:
        trip_id = trip['trip_id']
        # Assuming trip_truck_type and number_truck_type are also keys in the trip dictionary
        trip_truck_type = trip['trip_truck_type']
        number_truck_type = trip['number_truck_type']

        # Use a tuple to group trip data to avoid shuffling issues
        # Ensure trip_truck_type and number_truck_type are strings
        chromosome.append((trip_id, str(trip_truck_type), str(number_truck_type)))
    return chromosome

def initialize_population_trips(list_of_trips, population_size):
    """
    Initializes a population of chromosomes for the trips scenario.

    Args:
        list_of_trips: A list of dictionaries, where each dictionary represents a trip.
        population_size: The desired size of the population.

    Returns:
        A list of chromosomes representing the initial population.
    """
    initial_chromosome = initialize_chromosome_trips(list_of_trips)
    population = []

    for _ in range(population_size):
        # Shuffle the trips (tuples) within the chromosome
        chromosome_copy = initial_chromosome[:]
        random.shuffle(chromosome_copy)
        population.append(chromosome_copy)
    return population

def fitness_func_trips(chromosome, list_of_trips, trucks_csv_file):
    """
    Calculates the fitness value and sequence for a trips chromosome.
    Handles potential 'None' values in the chromosome to avoid errors.

    Args:
        chromosome: The chromosome representing a solution.
        list_of_trips: A list of dictionaries, where each dictionary represents a trip.
        trucks_csv_file: The path to the CSV file containing truck type costs.

    Returns:
        A tuple containing:
            - The fitness value (total cost) of the chromosome.
            - A list representing the sequence of truck types and numbers for each trip.
    """
    total_cost = 0
    truck_type_costs = {}
    sequence = []  # To store the truck types and numbers for each trip

    # Load truck type costs from CSV file
    with open(trucks_csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert the Truck_type to an integer if it's a string
            truck_type = row['Truck_type']  # No conversion to int here
            truck_type_costs[truck_type] = float(row['Cost'])

    # Calculate cost and build sequence for each trip
    for trip_data in chromosome:  # Iterate through the tuples
        trip_id, trip_truck_type, number_truck_type = trip_data

        # Handle potential 'None' values for number_truck_type
        try:
            number_truck_type = int(number_truck_type)
        except ValueError:
            number_truck_type = 1  # Set a default value (e.g., 1) if 'None'

        # Find the trip in the list_of_trips to get its distance
        # If trip_id is not found, use a default distance (e.g., 0)
        trip_distance = next((trip['trip_distance'] for trip in list_of_trips if trip['trip_id'] == trip_id), 0)

        # Calculate cost only if number_truck_type is greater than 0
        if number_truck_type > 0 and trip_truck_type in truck_type_costs:  # Check if key exists
            trip_cost = truck_type_costs[trip_truck_type] * trip_distance * number_truck_type
            total_cost += trip_cost

        # Add truck type and number to the sequence
        sequence.append((trip_id, trip_truck_type, number_truck_type))

    return total_cost, sequence



def select_parents_trips(population, list_of_trips, trucks_csv_file, tournament_size=3):
    """
    Selects two parent chromosomes from the population using tournament selection.

    Args:
        population: The current population of chromosomes.
        list_of_trips: A list of dictionaries, where each dictionary represents a trip.
        trucks_csv_file: The path to the CSV file containing truck type costs.
        tournament_size: The number of chromosomes to compete in each tournament.

    Returns:
        A list containing the two selected parent chromosomes.
    """
    parents = []
    for _ in range(2):  # Select 2 parents
        tournament = random.sample(population, tournament_size)  # Randomly select individuals for the tournament

        # Find the chromosome with the lowest fitness (total cost) in the tournament
        winner = min(tournament, key=lambda chromosome: fitness_func_trips(chromosome, list_of_trips, trucks_csv_file)[0])

        parents.append(winner)  # Add the winner to the parents list
    return parents



def crossover_trips(parent1, parent2):
    """
    Performs crossover between two parent chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A tuple containing the two offspring chromosomes.
    """
    # Check if the parents are long enough for crossover
    if len(parent1) < 6 or len(parent2) < 6:
        # If parents are too short, return them as offspring (no crossover)
        return parent1, parent2

    # Ensure crossover point is within trip boundaries (multiples of 3)
    crossover_point = random.randrange(3, len(parent1) - 3, 3)

    # Create offspring by swapping segments
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

    return offspring1, offspring2


def validate_and_repair_trips(offspring, list_of_trips, trucks_csv_file):
    """
    Validates and repairs an offspring chromosome, ensuring truck_type is not more than 3.

    Args:
        offspring: The offspring chromosome to validate and repair.
        list_of_trips: A list of dictionaries, where each dictionary represents a trip.

    Returns:
        The repaired offspring chromosome (the same object as the input).
    """
    repaired_data = []  # Temporary list to store repaired data
    trip_ids_present = set()  # Keep track of trip IDs already added

    # Load truck data for easy access
    with open(trucks_file, 'r') as file:
        reader = csv.DictReader(file)
        trucks_data = {row['Truck_type']: row for row in reader}

    # Helper function to calculate required trucks for a trip (similar logic to initialize_trips)
    def calculate_required_trucks(trip_details, truck_type, trucks_csv_file):
        with open(trucks_csv_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['Truck_type'] == truck_type:
                    truck_weight = float(row['Weight'])
                    truck_area = float(row['Area'])
                    return int(math.ceil(max(
                        trip_details['trip_weight'] / truck_weight,
                        trip_details['trip_area'] / truck_area
                    )))
            return 1  # Default to 1 if truck type not found

    # Helper function to get truck cost from trucks_csv_file
    def get_truck_cost(truck_type, trucks_csv_file):
        with open(trucks_csv_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['Truck_type'] == truck_type:
                    return float(row['Cost'])
            return 0.0  # Default to 0 if truck type not found

    # Create a set of all trip IDs from list_of_trips
    all_trip_ids = {trip['trip_id'] for trip in list_of_trips}

    # Iterate through offspring
    for trip_data in offspring:
        trip_id, trip_truck_type, number_truck_type_str = trip_data

        if trip_id in trip_ids_present:
            continue

        trip_ids_present.add(trip_id)

        trip_details = next((trip for trip in list_of_trips if trip['trip_id'] == trip_id), None)

        if trip_details:
            truck_details = trucks_data.get(trip_truck_type)

            if truck_details:
                best_truck_type = trip_truck_type
                best_num_trucks = number_truck_type_str  # Use number_truck_type_str directly
                min_cost = float('inf')

                if not (float(truck_details['Weight']) * int(number_truck_type_str) >= trip_details['trip_weight'] and
                        float(truck_details['Area']) * int(number_truck_type_str) >= trip_details['trip_area']):

                    for truck_type in ['1', '2', '3']:
                        num_trucks = calculate_required_trucks(trip_details, truck_type, trucks_csv_file)
                        cost = num_trucks * get_truck_cost(truck_type, trucks_csv_file)

                        if cost < min_cost:
                            min_cost = cost
                            best_truck_type = truck_type
                            best_num_trucks = num_trucks  # Update best_num_trucks with num_trucks

                # Append with best_num_trucks converted to string
                repaired_data.append((trip_id, best_truck_type, str(best_num_trucks)))

        else:
            repaired_data.append(trip_data)  # Append original trip_data if details not found
    # Find and fix missing trips
    missing_trip_ids = all_trip_ids - trip_ids_present
    for trip_id in missing_trip_ids:
        trip_details = next((trip for trip in list_of_trips if trip['trip_id'] == trip_id), None)

        if trip_details:
            # Find a suitable truck type and number based on trip requirements
            suitable_truck_type, num_trucks = find_suitable_truck(trip_details, trucks_csv_file)

            # Assign the optimized combination
            repaired_data.append((trip_id, suitable_truck_type, str(num_trucks)))

    offspring[:] = repaired_data
    return offspring

def find_suitable_truck(trip_details, trucks_csv_file):
    """
    Finds a suitable truck type and number based on trip requirements.
    """
    with open(trucks_csv_file, 'r') as file:
        reader = csv.DictReader(file)
        trucks_data = list(reader)  # Read all truck data into a list

        # Sort trucks by weight and area capacity (descending)
        trucks_data.sort(key=lambda truck: (float(truck['Weight']), float(truck['Area'])), reverse=True)

        for truck in trucks_data:
            num_trucks = int(math.ceil(max(
                trip_details['trip_weight'] / float(truck['Weight']),
                trip_details['trip_area'] / float(truck['Area'])
            )))
            if num_trucks > 0:
                return truck['Truck_type'], num_trucks

        # If no suitable truck found (highly unlikely), return largest truck type and num_trucks = 1
        return trucks_data[0]['Truck_type'], 1

def calculate_required_trucks(trip_details, truck_type, trucks_csv_file):
    """
    Calculates the required number of trucks for a trip, ensuring suitability.

    Args:
        trip_details (dict): Trip details including weight and area.
        truck_type (str): The type of truck.
        trucks_csv_file (str): Path to the trucks CSV file.

    Returns:
        int: The number of trucks required.
    """

    with open(trucks_csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Truck_type'] == truck_type:
                truck_weight = float(row['Weight'])
                truck_area = float(row['Area'])

                # Initial calculation (same as before)
                num_trucks = int(math.ceil(max(
                    trip_details['trip_weight'] / truck_weight,
                    trip_details['trip_area'] / truck_area
                )))

                # Verification step: Check if calculated trucks can handle the load
                total_weight_capacity = truck_weight * num_trucks
                total_area_capacity = truck_area * num_trucks

                # Adjust if capacity is insufficient
                while total_weight_capacity < trip_details['trip_weight'] or \
                      total_area_capacity < trip_details['trip_area']:
                    num_trucks += 1  # Increase truck count
                    total_weight_capacity = truck_weight * num_trucks
                    total_area_capacity = truck_area * num_trucks

                print(f"Trip weight: {trip_details['trip_weight']}, Trip area: {trip_details['trip_area']}")
                print(f"Truck weight: {truck_weight}, Truck area: {truck_area}")
                print(f"Calculated num_trucks: {num_trucks}")
                return num_trucks  # Return the adjusted or initial num_trucks

        return 1  # Default to 1 if truck type not found

def mutate_trips(chromosome, list_of_trips, trucks_csv_file, mutation_rate=0.01):
    """Applies mutation to a trips chromosome, ensuring truck_type and number_truck_type are valid."""

    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            # Mutate truck type and ensure it's suitable for the trip
            trip_id = chromosome[i][0]
            trip_details = next((trip for trip in list_of_trips if trip['trip_id'] == trip_id), None)

            if trip_details:
                suitable_truck_types = []
                with open(trucks_csv_file, 'r') as file:
                    reader = csv.DictReader(file)
                    for row in reader:
                        truck_type = row['Truck_type']
                        truck_weight = float(row['Weight'])
                        truck_area = float(row['Area'])

                        # Check if truck type can handle the trip's requirements
                        if truck_weight >= trip_details['trip_weight'] and truck_area >= trip_details['trip_area']:
                            suitable_truck_types.append(truck_type)

                if suitable_truck_types:
                    # Choose a random suitable truck type
                    new_truck_type = random.choice(suitable_truck_types)

                    # Calculate the required number of trucks for the chosen truck type
                    num_trucks = calculate_required_trucks(trip_details, new_truck_type, trucks_csv_file)

                    chromosome[i] = (chromosome[i][0], new_truck_type, str(num_trucks))

        # Mutation for the number of trucks is not needed as it's adjusted based on the truck type


def genetic_algorithm_trips(list_of_trips, trucks_csv_file, population_size=100, generations=100, mutation_rate=0.01):
    """
    Runs the genetic algorithm to find the best solution for the trips scenario.

    Args:
        list_of_trips: A list of dictionaries, where each dictionary represents a trip.
        trucks_csv_file: The path to the CSV file containing truck type costs.
        population_size: The size of the population.
        generations: The number of generations to evolve.
        mutation_rate: The probability of a gene being mutated.

    Returns:
        A tuple containing:
            - The best fitness value found.
            - The sequence (chromosome) corresponding to the best fitness value.
    """
    population = initialize_population_trips(list_of_trips, population_size)
    best_fitness = float('inf')  # Initialize with a very large value
    best_sequence = None

    for generation in range(generations):
        new_population = []

        # Create offspring using crossover and mutation
        for _ in range(population_size // 2):
            parent1, parent2 = select_parents_trips(population, list_of_trips, trucks_csv_file)
            offspring1, offspring2 = crossover_trips(parent1, parent2)

            # Pass trucks_csv_file to validate_and_repair_trips
            offspring1 = validate_and_repair_trips(offspring1, list_of_trips, trucks_csv_file)
            offspring2 = validate_and_repair_trips(offspring2, list_of_trips, trucks_csv_file)

            mutate_trips(offspring1, list_of_trips, trucks_csv_file, mutation_rate)
            mutate_trips(offspring2, list_of_trips, trucks_csv_file, mutation_rate)

            new_population.extend([offspring1, offspring2])

        # Replace the old population with the new population
        population = new_population

        # Find the best chromosome in the current generation
        current_best_chromosome = min(population, key=lambda chromosome: fitness_func_trips(chromosome, list_of_trips, trucks_csv_file)[0])
        current_best_fitness, current_best_sequence = fitness_func_trips(current_best_chromosome, list_of_trips, trucks_csv_file)

        # Update best fitness and sequence if a better solution is found
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_sequence = current_best_sequence

        # Print progress (optional)
        print(f"Generation {generation}: Best fitness = {best_fitness}")

    return best_fitness, best_sequence

def count_truck_types_detailed(best_sequence):
    """
    Counts the total number of each truck type in the best sequence
    and the sum total of all trucks assigned.

    Args:
        best_sequence: The best sequence (chromosome) representing the solution.

    Returns:
        dict: A dictionary containing the count of each truck type and the total.
    """
    truck_type_counts = {}
    total_trucks = 0

    for trip_data in best_sequence:
        truck_type = trip_data[1]  # Get the truck type
        num_trucks = int(trip_data[2])  # Get the number of trucks (convert to int)

        # Update the count for the truck type
        truck_type_counts[truck_type] = truck_type_counts.get(truck_type, 0) + num_trucks

        # Update the total number of trucks
        total_trucks += num_trucks

    truck_type_counts['Total number of trucks needed'] = total_trucks

    for truck_type, count in truck_type_counts.items():
        if truck_type != 'Total number of trucks needed':
            print(f"truck_type '{truck_type}'= {count}")
    print(f"Total number of trucks needed= {total_trucks}")

    return truck_type_counts

def print_trip_details(trip_id, list_of_trips):
    """
    Prints the source, destination, and distance for a given trip ID.

    Args:
        trip_id (str): The ID of the trip to search for.
        list_of_trips (list): The list of trip dictionaries.
    """
    for trip in list_of_trips:
        if trip['trip_id'] == trip_id:
            source = trip['source']
            destination = trip['destination']
            distance = trip['trip_distance']
            print(f"Trip {trip_id}:")
            print(f"  Source: {source}")
            print(f"  Destination: {destination}")
            print(f"  Distance: {distance}")
            print(f"  Total Trip Weight: {trip['trip_weight']}")
            print(f"  Total Trip Area: {trip['trip_area']}")

            return  # Stop searching once found

    print(f"Trip {trip_id} not found in the list of trips.")

# Load input data from CSV files
distance_file = "/content/distance.csv"  # Replace with your distance file path
trucks_file = "/content/trucks.csv"  # Replace with your trucks file path
orders_file = "/content/order_large.csv"  # Replace with your orders file path

# Initialize trips
list_of_trips = initialize_trips(distance_file, trucks_file, orders_file)

# Run the genetic algorithm
best_fitness, best_sequence = genetic_algorithm_trips(
    list_of_trips, trucks_file, population_size=100, generations=100, mutation_rate=0.01
)

# Get detailed truck type counts and total
detailed_counts = count_truck_types_detailed(best_sequence)


# Print the best fitness, sequence and detailed counts
print("Best Fitness:", best_fitness)
print("Best Sequence:", best_sequence)
print(detailed_counts)
#print_trip_details("trip_2", list_of_trips)

Trip trip_19:
  Source: City_61
  Destination: City_35
  Distance: 1728820.0
  Truck_Type: 1
